# Introduction à l'apprentissage automatique: TP4 - Exercice 2 - <font color=red> CORRECTION </font>

<br>

### Reconnaissance d'images par SVM: la base de données Fashion-MNIST

<br>

Dans cet exercice, on travaille sur la base de données Fashion-MNIST, qui a l'"avantage" de fournir un problème de classification plus difficile que la base MNIST du TP précédent.

Commencez par lire la description de la base ici: https://www.openml.org/d/40996

Pour limiter les temps de calcul pendant le TP, nous allons nous limiter aux 10000 premières images de la base. La cellule suivante charge les données et prépare une base d'apprentissage et une base de test.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, metrics, model_selection
%matplotlib inline 

# Fashion-Mnist database sur openML: (il faut une dizaine de secondes pour charger la base)
size_images=(28,28)
X_fashion, y_fashion = datasets.fetch_openml(data_id=40996, return_X_y=True, as_frame=False, parser='auto')
X_fashion=X_fashion[:10000,:]/255.  # normalisation des niveaux de gris entre 0 et 1
y_fashion=y_fashion[:10000]

for i in range(10):
    n=np.sum(y_fashion==str(i))
    print("nombre d'observations dans la classe %d: %d" %(i,n))

n_samples = len(X_fashion)
print("nombre total d'observations (apprentissage + test): %d" % n_samples)

n_features = len(X_fashion[0])
print("nombre de caractéristiques par observation: %d" % n_features)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_fashion, y_fashion, test_size=0.2, random_state=1)

print("nombre d'observations dans la base d'apprentissage: %d" %len(X_train))
print("nombre d'observations dans la base de test: %d" %len(X_test))


La cellule suivante définit une fonction qui permet d'afficher les 150 premières images de la base de test, ainsi que la classe $c$ déterminée par l'algorithme de classification et la classe véritable $c'$ sous la forme $c/c'$.

In [ ]:
def affichage_150_images(X_test,y_test,y_pred):
    plt.figure(figsize=[15,12])   
    for n in range(150):
        plt.subplot(10,15,n+1,xticks=[],yticks=[])
        plt.imshow(np.reshape(X_test[n,:],size_images),cmap='gray_r')
        if y_pred[n]==y_test[n]:
            plt.text(0.1,0.1,str(y_pred[n])+' / '+str(y_test[n]),fontsize=8,bbox=dict(facecolor='white', alpha=1))    
        else:
            plt.text(0.1,0.1,str(y_pred[n])+' / '+str(y_test[n]),fontsize=8,bbox=dict(facecolor='red', alpha=1))    
    plt.suptitle('classe predite / classe réelle')
    plt.show()

__Questions__:

Testez les classifieurs SVM à noyau linéaire et à noyau RBF. Pour vous éviter le temps de calcul de `GridSearchCV` (plusieurs minutes ici, étant donnée la taille de la base d'apprentissage):
- pour le noyau linéaire, vous utiliserez la valeur $C=0.1$ 
- pour le noyau RBF, vous utiliserez la valeur de $\gamma$ par défaut et $C=10$.

Comparez les scores de classification sur la base de test de ces classifieurs aux scores de l'algorithme du plus proche voisin, des 5 plus proches voisins, à la classification bayésienne naïve gaussienne, et à la régression logistique (voir l'exercice 2 du TP3). Remarquez les bonnes performances des classifieurs linéaires, la mauvaise performance de GNB, et expliquez-les par les éléments du cours.

Comparez également les temps de calcul: ajoutez `%time` (il s'agit d'une _magic command_ jupyter) devant les lignes où vous exécutez `fit` et `predict̀`). 

Vous obtenez l'indication Wall qui est la durée réellement écoulée sur une horloge murale (_wall clock_), ainsi que la valeur "CPU" qui compte le temps de calcul total sur le CPU (en additionnant le temps de processus s'exécutant en parallèle sur les coeurs du processeurs). Comme tous les modèles de prédiction de `scikit-learn` ne bénéficient pas de la parallélisation sur plusieurs coeurs, seuls les temps d'exécution "CPU" sont réellement comparables.


Vous lirez dans la correction (à venir) une discussion sur le rôle des vecteurs supports (__facultatif__). 

In [ ]:
from sklearn import svm, model_selection, neighbors, naive_bayes, linear_model

In [ ]:
# facultatif: gridsearch pour SVM linear
# attention, peut prendre plusieurs minutes (on se contente des valeurs de l'énoncé pendant le TP)

C_range=10**(np.arange(-2.,2.5,1)) 
parameters = {'C': C_range }
SVM1 = svm.SVC(kernel='linear')
gridsearch1 = model_selection.GridSearchCV(SVM1, parameters,n_jobs=-1)
%time gridsearch1.fit(X_train,y_train)
print("Meilleur estimateur trouvé:")
print(gridsearch1.best_estimator_)
print("Meilleur paramètre:")
print(gridsearch1.best_params_)

In [ ]:
# facultatif: gridsearch pour SVM RBF
# attention, peut prendre plusieurs minutes (on se contente des valeurs de l'énoncé pendant le TP)

C_range=10**(np.arange(-2.,2.5,1)) 
parameters = {'C': C_range }
SVM2 = svm.SVC(kernel='rbf')
gridsearch2 = model_selection.GridSearchCV(SVM2, parameters,n_jobs=-1)
%time gridsearch2.fit(X_train,y_train)
print("Meilleur estimateur trouvé:")
print(gridsearch2.best_estimator_)
print("Meilleur paramètre:")
print(gridsearch2.best_params_)

In [ ]:
# SVM linear
print('SVM linear\n')
SVM1 = svm.SVC(kernel='linear',C=0.1)
%time SVM1.fit(X_train, y_train)
%time y_pred_svm1 = SVM1.predict(X_test)

print('SVM1 score: %f' % metrics.accuracy_score(y_test, y_pred_svm1))

affichage_150_images(X_test,y_test,y_pred_svm1)        

print(metrics.classification_report(y_test,y_pred_svm1))

print(metrics.confusion_matrix(y_test,y_pred_svm1))

In [ ]:
# SVM rbf
print('SVM rbf\n')
SVM2 = svm.SVC(kernel='rbf', C=10)
%time SVM2.fit(X_train, y_train)
%time y_pred_svm2 = SVM2.predict(X_test)

print('SVM2 score: %f' % metrics.accuracy_score(y_test, y_pred_svm2))

affichage_150_images(X_test,y_test,y_pred_svm2)        

print(metrics.classification_report(y_test,y_pred_svm2))

print(metrics.confusion_matrix(y_test,y_pred_svm2))

In [ ]:
# classification au plus proche voisin
knn = neighbors.KNeighborsClassifier(n_neighbors=1, n_jobs=-1) 
%time knn.fit(X_train, y_train)
%time y_pred_nn = knn.predict(X_test)

print('KNN1 score: %f' % metrics.accuracy_score(y_test, y_pred_nn))

affichage_150_images(X_test,y_test,y_pred_nn)        

print(metrics.classification_report(y_test,y_pred_nn))

print(metrics.confusion_matrix(y_test,y_pred_nn))

In [ ]:
# classification aux 5 plus proche voisins
knn5 = neighbors.KNeighborsClassifier(n_neighbors=5, n_jobs=-1) 
%time knn5.fit(X_train, y_train)
%time y_pred_nn5 = knn5.predict(X_test)

print('KNN5 score: %f' % metrics.accuracy_score(y_test, y_pred_nn5))

affichage_150_images(X_test,y_test,y_pred_nn5)        

print(metrics.classification_report(y_test,y_pred_nn5))

print(metrics.confusion_matrix(y_test,y_pred_nn5))

In [ ]:
# classifieur naif de Bayes gaussien:
NB = naive_bayes.GaussianNB()
%time NBfit=NB.fit(X_train, y_train)
%time y_pred_nb = NBfit.predict(X_test)

print('score GNB: %.3f' % metrics.accuracy_score(y_test, y_pred_nb))
    
affichage_150_images(X_test,y_test,y_pred_nb)

print(metrics.classification_report(y_test,y_pred_nb))

print(metrics.confusion_matrix(y_test,y_pred_nb))


In [ ]:
# régression logistique
LR = linear_model.LogisticRegression(max_iter=5000, penalty=None)
%time LR.fit(X_train, y_train)
%time y_pred_lr=LR.predict(X_test)

print('score LR: %3f' % metrics.accuracy_score(y_test, y_pred_lr))
    
affichage_150_images(X_test,y_test,y_pred_lr)

print(metrics.classification_report(y_test,y_pred_lr))

print(metrics.confusion_matrix(y_test,y_pred_lr))

<font color=red>
    
### Commentaires:
 
On attend en réponse à cette question une discussion qualitative. On indique les scores sur la base test.
    
Les temps indiqués dépendent bien entendu du processeur.
    
Pour résumer (sur ma machine CPU Intel Core i7 @ 1.80GHz and 16 Gb memory):
    
### 1PPV

entraînement Wall time: 9.52 ms

prédiction Wall time: 615 ms

KNN1 score: 0.816000

### 5PPV
 
entraînement Wall time: 25.1 ms
    
prédiction Wall time: 875 ms
    
KNN5 score: 0.833000
    
### GNB

entraînement Wall time: 120 ms
    
prédiction Wall time: 170 ms
    
score GNB: 0.529
  
### LR

entraînement Wall time: 23.2 s
    
prédiction Wall time: 5.02 ms
    
score LR: 0.770000
   
### SVM linear
 
entraînement Wall time: 6.27 s
    
prédiction Wall time: 3.19 s
    
SVM1 score: 0.849500
    
### SVM RBF
    
entraînement Wall time: 8.12 s
    
prédiction Wall time: 28.1 s
    
SVM2 score: 0.878500

<br>
    
Tout d'abord on constate que les classifieurs linéaires (SVM linear et LR) donnent de bonnes performances: cela correspond à la situation décrite dans le polycopié d'un problème de classification en grande dimension, pour lequel il est "assez facile" de séparer linéairement les classes. SVM linear donne un score meilleur que LR. Néanmoins, on voit que le classifieur non-linéaire SVM RBF permet une augmentation du score de prédiction. GNB a une performance très modeste: l'hypothèse d'indépendance conditionnelle des caractéristiques est trop grossière. En effet, les caractéristiques sont les valeurs des niveaux de gris en chaque pixel: on suppose donc dans GNB que, dans chaque classe, les pixels ne sont pas "liés" entre eux. Cette hypothèse est bien entendu en contradiction flagrante avec le fait que c'est la perception de groupes de pixels agencés les uns par rapport aux autres qui nous fait reconnaître une forme.
    
<br>
    
  
Pour les deux SVM: 
- la classe 6 (chemises) a la moins bonne précision: certains vêtements sont reconnus comme des chemises à tort.
- la classe 6 a également le moins bon rappel: certaines chemises sont reconnus comme d'autres vêtements à tort.

On retrouve ce constat sur les matrices de confusion: 
- la colonne de 6 montre que des 0 (t-shirts) et 2 (pull-over) sont reconnus comme chemises
- la ligne de 6 montre que des chemises sont reconnus comme des 0 (t-shirts), 2 (pull-over), 4 (manteau).
    
On voit sur la sortie de `affichage_150_images` que les erreurs correspondent effectivement à des cas où l'image est assez ambigüe.
  
<br> 
     
_Complément_: Le temps d'entraînement des SVM est plus faible que pour LR (d'autant plus que LR est parallélisé sur tous les coeurs, contrairement aux SVM), mais la prédiction est bien plus rapide pour LR. La prédiction par LR consiste essentiellement à calculer un produit scalaire en dimension 28x28 par classe, alors que la prédiction par SVM nécessite de calculer pour chaque paire de classes (classification multi-classe one-vs-one pour SVC) une somme de $k(x_i,x)$ avec les $x_i$ vecteurs supports, ce qui est plus lourd dans cette expérience (ce n'est pas une vérité générale).

 
</font>

## Discussion sur les vecteurs supports (facultatif)

In [ ]:
print('nombre de vecteurs supports par classe et coefficients du dual:') 
print('\nSVM linear')
print('\nnombre de vecteurs supports par classe:')
print(SVM1.n_support_)
print('\ncoefficient duaux:')
print(SVM1.dual_coef_)
print('\ndimension de la matrice des coefficients duaux:')
print(SVM1.dual_coef_.shape)
print('\nnombre total de vecteurs supports')
print(np.sum(SVM1.n_support_))
print('\n\nSVM RBF')
print('\nnombre de vecteurs supports:')
print(SVM2.n_support_)
print("\ncoefficient duaux (il s'agit de la valeur de y_i * lambda_i):")
print(SVM2.dual_coef_)
print('\ndimension de la matrice des coefficients duaux:')
print(SVM2.dual_coef_.shape)
print('\nnombre total de vecteurs supports')
print(np.sum(SVM2.n_support_))

<font color=red>

__Complément facultatif: à propos des vecteurs supports__
    
<br>

    
Le noyau RBF fournit plus de vecteurs supports que le noyau linear. Cela explique les temps de prédiction plus longs pour RBF que pour linear, et peut-être aussi le temps d'entraînement plus long (mais cela dépend de l'algorithme d'optimisation utilisé: le nombre de vecteurs supports joue un rôle mais difficile d'en dire davantage).
    
<br>
    
Attention, le contenu de `dual_coef_` de `scikit-learn` est en fait la valeur de $y_i \lambda_i$ dans le cours: les coefficients duals sont positifs par définition: une valeur négative dans la matrice `dual_coef_` signifie que le vecteur support a été associé à la classe -1.
    
On voit que les vecteurs supports ne sont pas répartis uniformément entre les classes. Globalement les classes 0, 2, 4, 6 nécessitent le plus de vecteurs support, et 1, 7, 8, 9 le moins de vecteurs supports (pour les deux noyaux). Intuitivement, si une classe nécessite beaucoup de vecteurs supports, c'est qu'elle est difficile à discriminer d'au moins une autre classe, il  faut alors la représenter par plus de vecteurs (rappelons que les vecteurs "non-supports" n'interviennent pas dans la prédiction). Il s'agit d'une discussion assez heuristique mais on peut aussi remarquer que les classes 2,4,6 ont le moins bon f1-score (elles semblent effectivement difficile à représenter), et la classe 1 le meilleur f1-score.
    
Si on est curieux, on peut se demander ce que sont les vecteurs supports dans un problème de classification multiclasses. On sait qu'en fait c'est une stratégie one-vs-one qui est implantée par `SVC`: une SVM est entraînée par couple de classes. Chaque classe doit donc avoir 9 listes de vecteurs supports (une liste par classe alternative). L'attribut `dual_coef_` contient la liste des 9 coefficients duals (duaux ?) des vecteurs supports: 9 car chaque coefficient est fourni par une SVM one-vs-one. Les vecteurs supports comptés par `n_support` sont ceux qui sont vecteurs supports d'__au moins une SVM__, pour lequel le coefficient dual correspondant est non nul. On voit qu'une observation n'est généralement pas vecteur support de plus d'une SVM (le coefficient dual est généralement nul sauf dans un cas). 
    
</font>